In [1]:
# `setup.jl` will load core packages including the ED package
include("./src/setup.jl")

┌ Warning: PyPlot is using tkagg backend, which is known to cause crashes on MacOS (#410); use the MPLBACKEND environment variable to request a different backend.
└ @ PyPlot /Users/newberry/.julia/packages/PyPlot/XHEG0/src/init.jl:192


⊗ (generic function with 1 method)

# Generating Ainf data

In [2]:
L_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
jz_list = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.3]
jx = 1.0
g = 0.3

# Logarithmic time scale for Ainf, default from 0.1 to 10000
t_list = 10.0 .^(range(-1,stop=4,length=200))

Ainfs_dict = Dict()

for (L_i, L) in enumerate(L_list)
    println(L)
    for (jz_i, jz) in enumerate(jz_list)
        for (gamma_i, gamma) in enumerate(gamma_list)
            Ainf = ED.get_Ainf(L, g, jx, jz, gamma, t_list)
            Ainfs_dict[L, jz, gamma] = Ainf
        end
    end
end

2
3d: ti = 200
4d: ti = 200


Excessive output truncated after 524301 bytes.

In [12]:
L_list = [2, 3, 4, 5, 6, 7, 8, 9, 10]
L_list = [11, 12]
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
jz_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
jx = 1.0
g_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]

# Logarithmic time scale for Ainf, default from 0.1 to 10000
t_list = 10.0 .^(range(-1,stop=4,length=200))

Ainfs_dict = Dict()

for (L_i, L) in enumerate(L_list)
    println(L)
    for (jz_i, jz) in enumerate(jz_list)
        for (gamma_i, gamma) in enumerate(gamma_list)
            for (g_i, g) in enumerate(g_list)
                Ainf = ED.get_Ainf(L, g, jx, jz, gamma, t_list)
                Ainfs_dict[L, jz, gamma, g] = Ainf
            end
        end
    end
end

2


Excessive output truncated after 524297 bytes.

In [13]:
Ainfs_dict

Dict{Any, Any} with 4536 entries:
  (10, 0.2, 0.4, 0.1) => [0.997208, 0.996866, 0.996483, 0.996054, 0.995572, 0.9…
  (3, 0.7, 0.6, 0.5)  => [0.98452, 0.982637, 0.980528, 0.978165, 0.97552, 0.972…
  (6, 0.1, 0.3, 0.9)  => [0.981269, 0.978988, 0.976431, 0.973566, 0.970356, 0.9…
  (4, 0.4, 0.3, 0.1)  => [0.994173, 0.993461, 0.992663, 0.991767, 0.990764, 0.9…
  (3, 0.4, 0.3, 0.7)  => [0.984659, 0.982792, 0.9807, 0.978356, 0.97573, 0.9727…
  (3, 0.3, 0.2, 0.2)  => [0.994221, 0.993515, 0.992723, 0.991835, 0.990839, 0.9…
  (8, 0.0, 0.7, 0.2)  => [0.998754, 0.998602, 0.998431, 0.998239, 0.998024, 0.9…
  (4, 0.9, 0.8, 0.2)  => [0.982908, 0.980826, 0.978492, 0.975878, 0.972948, 0.9…
  (6, 0.4, 0.8, 0.1)  => [0.996413, 0.995975, 0.995483, 0.994932, 0.994314, 0.9…
  (5, 0.5, 0.8, 0.3)  => [0.993035, 0.992186, 0.991233, 0.990165, 0.988968, 0.9…
  (7, 0.1, 0.3, 0.2)  => [0.996559, 0.996138, 0.995666, 0.995136, 0.994542, 0.9…
  (6, 0.4, 0.1, 0.7)  => [0.983072, 0.981012, 0.978704, 0.976118, 0.973222,

In [14]:
using JLD

save("data/Ainfs_dict_low_gamma_g.jld", 
    "Ainfs_dict", Ainfs_dict, 
    "t_list", t_list, 
    "L_list", L_list, 
    "jz_list", jz_list,
    "gamma_list", gamma_list, 
    "jx_list", [1.0],
    "g_list", g_list)

In [ ]:
test = load( "Ainfs_dict_v2.jld"  )
test["Ainfs_dict"] == Ainfs_dict

# Generating Ainf data 2

In [5]:
ED.get_Ainf

get_Ainf (generic function with 1 method)

In [ ]:
function get_Ayy_inf(L::Int64,g::Float64,jx::Float64,jz::Float64, gamma::Float64,times::AbstractArray)
    #=
    Unoptimized autocorrelation for y1-y1
    =#
    ainfs = zeros(Complex{Float64}, length(times))
    sigx = sigX(L,1)
    sigz = sigZ(L,1)
    sigy = im.*(sigz*sigx)
    us,vs = eigen(H_Delta(L,jx,jz,g,gamma))
    sigyV = vs'*sigy*vs
    for ti in 1:length(times)
        print("ed: ti = $ti\r")
        U = Diagonal(exp.(-im*times[ti].*us))
        ainfs[ti] = 1/(2^L)*tr(U'*sigyV*U*sigyV)
    end
    return real.(ainfs)
end

function get_Ainf(L::Int64,g::Float64,jx::Float64,jz::Float64, gamma::Float64,times::AbstractArray)
    #=
    This is partially optimized.
    =#
    ainfs = zeros(Complex{Float64}, length(times))
    sigx = Complex{Float64}.(sigX(L,1))
    us,vs = get_uvs(L,g,jx,jz,gamma)
    sigxV = vs'*sigx*vs
    for ti in 1:length(times)
        print("ed: ti = $ti\r")
        U = Diagonal(exp.(-im*times[ti].*us))
        ainfs[ti] = 1/(2^L)*tr(U'*sigxV*U*sigxV)
    end
    return real.(ainfs)
end

In [14]:
L_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
jz_list = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.3]
jx = 1.0
g = 0.3

# Logarithmic time scale for Ainf, default from 0.1 to 10000
t_list = 10.0 .^(range(-1,stop=4,length=200))

Ainfs_dict = Dict()

for (L_i, L) in enumerate(L_list)
    println(L)
    for (jz_i, jz) in enumerate(jz_list)
        for (gamma_i, gamma) in enumerate(gamma_list)
            Ainf = ED.get_Ayy_inf(L, g, jx, jz, gamma, t_list)
            Ainfs_dict[L, jz, gamma] = Ainf
        end
    end
end

2
3d: ti = 200
4d: ti = 200


Excessive output truncated after 524301 bytes.

In [12]:
using JLD

In [13]:
save("data/Ainfs_dict_sigy_v2.jld", 
    "Ainfs_dict", Ainfs_dict, 
    "t_list", t_list, 
    "L_list", L_list, 
    "jz_list", jz_list,
    "gamma_list", gamma_list, 
    "jx", 1.0,
    "g", g)

In [9]:
test = load( "Ainfs_dict_sigy_v1.jld"  )

Dict{String, Any} with 7 entries:
  "g"          => 0.3
  "jz_list"    => [0.0, 0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.3]
  "L_list"     => [2, 3, 4, 5, 6, 7, 8, 9]
  "jx"         => 1.0
  "Ainfs_dict" => Dict{Any, Any}((4, 0.3, 0.2)=>[0.989245, 0.987932, 0.98646, 0…
  "gamma_list" => [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  "t_list"     => [0.1, 0.105956, 0.112267, 0.118953, 0.126038, 0.133545, 0.141…

In [10]:
test["Ainfs_dict"] == Ainfs_dict

true